<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Nahranie-potrebných-knižníc" data-toc-modified-id="Nahranie-potrebných-knižníc-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Nahranie potrebných knižníc</a></span></li><li><span><a href="#Pôvodný-kód" data-toc-modified-id="Pôvodný-kód-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pôvodný kód</a></span><ul class="toc-item"><li><span><a href="#Definovanie-potrebných-parametrov" data-toc-modified-id="Definovanie-potrebných-parametrov-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Definovanie potrebných parametrov</a></span></li><li><span><a href="#Simulácia-hry" data-toc-modified-id="Simulácia-hry-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Simulácia hry</a></span></li><li><span><a href="#Výsledky" data-toc-modified-id="Výsledky-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Výsledky</a></span></li></ul></li><li><span><a href="#Sofistikovanejšia-verzia" data-toc-modified-id="Sofistikovanejšia-verzia-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sofistikovanejšia verzia</a></span><ul class="toc-item"><li><span><a href="#Definícia-funkcie,-ktorá-bude-hru-simulovať" data-toc-modified-id="Definícia-funkcie,-ktorá-bude-hru-simulovať-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Definícia funkcie, ktorá bude hru simulovať</a></span></li><li><span><a href="#Test,-či-výsledky-korešpondujú-s-výsledkami-v-článku" data-toc-modified-id="Test,-či-výsledky-korešpondujú-s-výsledkami-v-článku-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Test, či výsledky korešpondujú s výsledkami v článku</a></span></li><li><span><a href="#Vygenerovanie-rôznych-simulácií" data-toc-modified-id="Vygenerovanie-rôznych-simulácií-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Vygenerovanie rôznych simulácií</a></span></li></ul></li></ul></div>

# Nahranie potrebných knižníc

In [ ]:
# Knižnice
import random as rand
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

# 1. Primitívna verzia
## 1.1. Definovanie potrebných parametrov

In [ ]:
# Pocet simulacii
S = 100_000
T_points = 10  # Doplnit
# Pocet hier, kedy treba zastavit
g_stop = 6      # Doplnit
# Pocet vyhier Alice, pri ktorej sa zastavime
A_stop = 1
# Pocitadlo stavov
r = 0
# Stredna hodnota vyhry boba
E_Bwin = 0

## 1.2. Simulácia hry

In [ ]:
# Seed
rand.seed(123)
for s in range(0, S):
    # Hodenie gule po stole
    p = np.random.uniform()
    # Zahrame g_stop hier - vektor s dlzkou 1 x g_stop
    y = np.random.uniform(size=g_stop)
    # Pocet vyhier Alice a Boba
    A_w = sum(y < p)
    B_w = len(y) - A_w
    # Stav kedy pocet vyhier Alice == A_stop a celkovy pocet hier == g_stop
    if A_w == A_stop:
        # Pripocitaj, ze nastal nami zvoleny stav
        r += 1
        # Pri tomto preruseni hru dohrame a zistim, kto by vyhral
        while A_w < T_points and B_w < T_points:
            y2 = rand.random()
            if y2 < p:
                A_w += 1
            else:
                B_w += 1
        # Ak je pocet bodov Boba vyssi ako Alice, tak mu pripocitame vitazstvo
        if A_w < B_w:
            E_Bwin += 1

## 1.3. Výsledky 

*   Test, či výsledok korešponduje s výsledkom v článku / na cvičení



In [ ]:
# pocet bodov Boba pri stopnuti
B_stop = g_stop - A_stop
# Result string (Bob je prvy)
res = str(B_stop) + ":" + str(A_stop)
# Ocakavana vyhra Boba, Alice, Ferovy podil sanci z pohladu Boba
e_bwin = round(E_Bwin / r, 3)
e_awin = round(1 - e_bwin, 3)
odds = round(e_bwin / e_awin, 1)
res_prop = round(r / S, 3)

print(f"Vitazny pocet bodov: {T_points}\n")
print(f"Zastavenie po {g_stop} odohranych hrach\n")
print(f"Vysledok pri stopntuti: {res}\n")
print(f"E[Bob=W]: {e_bwin}\n")
print(f"E[Alice=W]: {e_awin}\n")
print(f"Ferovy podil sanci: {odds}\n")
print(f"Pocet stavov: {r}\n")
print(f"Relativny pocet stavov: {res_prop}\n")

Vitazny pocet bodov: 10

Zastavenie po 6 odohranych hrach

Vysledok pri stopntuti: 5:1

E[Bob=W]: 0.972

E[Alice=W]: 0.028

Ferovy podil sanci: 34.7

Pocet stavov: 14210

Relativny pocet stavov: 0.142



# 2. Sofistikovanejšia verzia
## 2.1. Definícia funkcie, ktorá bude hru simulovať

In [ ]:
def a_b_game(p_to_win=6, g_stop=8, a_points=5, S=100_000):
    """
    Vypocita pravdepodobnosti vyhry Boba a Alice podla zadanych parametrov:
    p_to_win -> Pocet bodov do vitazstva
    g_stop   -> V akom bode hru zastavime
    a_points -> Body Alice, ktore nam definuju pozadovany stav v bode zastavenia
    S        -> Pocet simulacii
    """
    # Import kniznice pre generovanie nahodnych cisel
    import random as rand
    # Nastavenie pocitadiel
    r = 0
    E_Bwin = 0
    # Upozornenie velmi velky na pocet simulacii
    if S > 10_000_000:
        print("Pozor!!! Toto moze tvojmu pocitacu zabrat vela casu.")
    # Stanovenie niekolkych podmienok, ktore by nemali byt porusene
    cond1 = g_stop > ((p_to_win * 2) - 2)
    cond2 = ((p_to_win - a_points) <= 0)
    cond3 = cond1 and cond2
    cond4 = ((g_stop - a_points) > p_to_win) or a_points > g_stop
    # Zakomponovanie podmienok a prislusnych upozorneni
    if cond1:
        return print("Zastavenie nastava v bode, kedy je o vitazstve rozhodnute!"
                    "Zmen parameter: 'g_stop'.")
    elif cond2:
        return print("Pocet bodov Alice je prilis velky! O vysledku je v tomto pripade rozhodnute!"
                     "\nZmen parameter: 'a_points'.")
    elif cond3:
        return print("Zastavenie je po prilis vela hrach a pocet bodov Alice je prilis vysoky!"
                     " O vitazstve je tym padom rozhodnute!\n Zmen parametre: 'g_stop' & 'a_points' ...")
    elif cond4:
        return print("Pocet bodov Alice je prilis maly/velky alebo sa hra zastavila prilis skoro/neskoro!"
                     " O vitazstve je tym padom rozhodnute!\n Zmen parameter: 'a_points' alebo 'g_stop ...'")
    else:
        for s in range(0, S):
            # Hodenie gule po stole
            p = np.random.uniform()
            # Zahrame g_stop hier - vektor s dlzkou 1 x g_stop
            y = np.random.uniform(size=g_stop)
            # Pocet vyhier Alice a Boba
            A_w = sum(y < p)
            B_w = len(y) - A_w
            # Stav kedy pocet vyhier Alice = a_points a celkovy pocet hier = g_stop
            if A_w == a_points:
                # Pripocita ak dany stav nastal
                r += 1
                # Pri tomto preruseni hru dohrame a zistime, kto by vyhral
                while A_w < p_to_win and B_w < p_to_win:
                    y2 = rand.random()
                    if y2 < p:
                        A_w += 1
                    else:
                        B_w += 1
                # Ak je pocet bodov Boba vyssi ako Alice, tak mu pripocitame vitazstvo
                if A_w < B_w:
                    E_Bwin += 1
        # pocet bodov Boba pri stopnuti
        b_points = g_stop - a_points
        # Vysledok v stopnuti (Bob je prvy)
        res = str(b_points) + ":" + str(a_points)
        # Ocakavana vyhra Boba, Alice, Ferovy podil sanci z pohladu Boba
        e_bwin = round(E_Bwin / r, 3)
        e_awin = round(1 - e_bwin, 3)
        odds = round(e_bwin / e_awin, 3)
        # Proporcia zadefinovanych stavov na celkovom pocte simulacii
        res_prop = round(r / S, 3)
        dict_df = {"Body do víťazstva": p_to_win,
                   "Počet hier": g_stop,
                   "Výsledok pri 'zastavení'": res,
                   "E[Bob = Výhra]": e_bwin,
                   "E[Alice = Výhra]": e_awin,
                   "Férový podiel šancí [Bob]": odds,
                   "Početnosť zadaných stavov": r,
                   "Rel. početnosť zadaných stavov": res_prop
                   }
        # Vrati dictionary, ktora sa pouzije na prezentovanie vysledkov
        return dict_df

##  2.2. Test, či výsledok korešponduje s výsledkom v článku / na cvičení

In [ ]:
res = a_b_game(6, 8, 5, 100_000)
res

{'Body do víťazstva': 6,
 'E[Alice = Výhra]': 0.909,
 'E[Bob = Výhra]': 0.091,
 'Férový podiel šancí [Bob]': 0.1,
 'Počet hier': 8,
 'Početnosť zadaných stavov': 11084,
 'Rel. početnosť zadaných stavov': 0.111,
 "Výsledok pri 'zastavení'": '3:5'}

## 2.3. Vygenerovanie rôznych simulácií a prezentovanie výsledkov

In [ ]:
results = []
for tp, gs, ap in zip([4, 5, 6, 7, 8, 9, 10], [0, 5, 3, 7, 6, 5, 12], [0, 2, 2, 3, 4, 2, 3]):
    results.append(a_b_game(tp, gs, ap, 100_000))

pd.DataFrame(results)

,Body do víťazstva,Počet hier,Výsledok pri 'zastavení',E[Bob = Výhra],E[Alice = Výhra],Férový podiel šancí [Bob],Početnosť zadaných stavov,Rel. početnosť zadaných stavov
0,4,0,0:0,0.501,0.499,1.004,100000,1.000
1,5,5,3:2,0.742,0.258,2.876,16589,0.166
2,6,3,1:2,0.269,0.731,0.368,25094,0.251
3,7,7,4:3,0.702,0.298,2.356,12511,0.125
4,8,6,2:4,0.155,0.845,0.183,14341,0.143
5,9,5,3:2,0.691,0.309,2.236,16702,0.167
6,10,12,9:3,0.998,0.002,499.000,7644,0.076


In [ ]:
# Vytvorenie HTML formatu
%%shell
jupyter nbconvert --to html /content/du_01_ukol_01.ipynb

[NbConvertApp] Converting notebook /content/du_01_ukol_01.ipynb to html
[NbConvertApp] Writing 306921 bytes to /content/du_01_ukol_01.html
